In [1]:
# snnTorch Imports
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen
from snntorch import surrogate
from snntorch import functional as SF

# Torch Imports
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Python Libary Imports
import matplotlib.pyplot as plt
import numpy as np
import itertools
from snntorch import utils
import random
import configparser

# Script Imports
from test_config import getHyperParameters

In [2]:
#network parameters
nin = 2
nout = 1
rand_range_start = 1
rand_range_end = 10
beta = 0.95
spike_grad = surrogate.fast_sigmoid(slope=25)

In [ ]:
class EONS:
    def __init__(self,params):
        self.params = params
        self.template_network = None
        pass
    def make_template_network(self,nin:int,nout:int):
        hidden = random.randint(rand_range_start,rand_range_end)
        hidden2 = random.randint(rand_range_start,rand_range_end)
        hidden3 = random.randint(rand_range_start,rand_range_end)
        net = nn.Sequential(nn.Linear(nin,hidden),
                            snn.Leaky(beta=beta,spike_grad=spike_grad),
                            nn.Linear(hidden,hidden2),
                            snn.Leaky(beta=beta,spike_grad=spike_grad),
                            nn.Linear(hidden2,hidden3),
                            snn.Leaky(beta=beta,spike_grad=spike_grad),
                            nn.Linear(hidden3,nout),
                            snn.Leaky(beta=beta,spike_grad=spike_grad))
        self.template_network = net
        return None
    
    def add_node(self):
        """
        This method adds a neuron to a randomly selected hidden layer and updates the 
        in_features to the next layer accordingly.
        """
        # Don't do anything if the network didn't initialize properly
        if self.template_network is None:
            print("Template network not initialized")
            return
        
        # Find indicies for all linear layers
        layer_indices = [i for i, layer in enumerate(self.template_network) if isinstance(layer, nn.Linear)]

        # Exclude input and output layers
        if (len(layer_indices) <= 2):
               print("Not enough layers to modify (only input/output layers present)")

        hidden_layer_indices = layer_indices[1:-1] # Only hidden layers
        selected_index = random.choice(hidden_layer_indices) # Select random layer from hidden layer list
        next_index = layer_indices[layer_indices.index(selected_index) + 1] # Get the next layer so that we can update the input

        # Get the layers
        linear = self.template_network[selected_index]
        next_linear = self.template_network[next_index]

        in_feat= linear.in_features # Keeping in features the same
        new_out_feat = linear.out_features + 1 # Adding neuron to the out

        # Creating updated hidden layer
        new_linear = nn.Linear(in_feat, new_out_feat)

        # Ensuring it keeps the same weights and biases
        with torch.no_grad():
             new_linear.weight[:linear.out_features] = linear.weight # Copies weights from old layer to new one
             new_linear.bias[:linear.out_features] = linear.bias # Copies bias value

        # Repeat for the next hidden layer. This ensures that the output of prev layer matches input of next layer
        next_out = next_linear.out_features
        new_next_linear = nn.Linear(new_out_feat, next_out)
        with torch.no_grad():
            new_next_linear.weight[:, :linear.out_features] = next_linear.weight # Copies only the part of the next layer's weight matrix that connects to the existing neurons
            new_next_linear.bias = next_linear.bias # Copies biases for output layer neuron

        # Replacing both layers in the sequential
        layers = list(self.template_network)
        layers[selected_index] = new_linear
        layers[next_index] = new_next_linear
        
        self.template_network = nn.Sequential(*layers)
        print(f"Added a neuron to output of layer {selected_index} and updated input to layer {next_index}")

    def remove_node(self):
        """
        This method removes a neuron from a randomly selected hidden layer.
        The in_features of the next layer is updated accordingly.
        If there isn't enough neurons in the layer to remove (if # of neurons is <= 1),
        the function simply returns and no mutation occurs. 
        """
        # Don't do anything if the network didn't initialize properly
        if self.template_network is None:
            print("Template network not initialized")
            return
        
        # Find indicies for all linear layers
        layer_indices = [i for i, layer in enumerate(self.template_network) if isinstance(layer, nn.Linear)]

        # Exclude input and output layers
        if (len(layer_indices) <= 2):
               print("Not enough layers to modify (only input/output layers present)")

        hidden_layer_indices = layer_indices[1:-1] # Only hidden layers
        selected_index = random.choice(hidden_layer_indices) # Select random layer from hidden layer list
        next_index = layer_indices[layer_indices.index(selected_index) + 1] # Get the next layer so that we can update the input

        # Get the layers
        linear = self.template_network[selected_index]
        next_linear = self.template_network[next_index]

        # Only continue if there are more than one neuron in the out features
        if (linear.out_features <= 1):
            print(f"Not enough neurons in layer {selected_index}. Cannot remove neuron.")
            return

        in_feat= linear.in_features # Keeping in features the same
        new_out_feat = linear.out_features - 1 # Removing neuron to the out

        # Creating updated hidden layer
        new_linear = nn.Linear(in_feat, new_out_feat)

        # Ensuring it keeps the same weights and biases
        with torch.no_grad():
             new_linear.weight[:] = linear.weight[:new_out_feat] # Keep the first N-1 neurons' weights
             new_linear.bias[:] = linear.bias[:new_out_feat] # Copies bias values except the one being removed

        # Repeat for the next hidden layer. This ensures that the output of prev layer matches input of next layer
        next_out = next_linear.out_features
        new_next_linear = nn.Linear(new_out_feat, next_out)
        with torch.no_grad():
            new_next_linear.weight[:, :] = next_linear.weight[:, :new_out_feat] # Copies all output rows, but only the first new_out_feat input columns
            new_next_linear.bias = next_linear.bias # Copies biases for output layer neuron

        # Replacing both layers in the sequential
        layers = list(self.template_network)
        layers[selected_index] = new_linear
        layers[next_index] = new_next_linear
        
        self.template_network = nn.Sequential(*layers) # Update network
        print(f"Removed a neuron to output of layer {selected_index} and updated input to layer {next_index}")

    def update_node_param(self):
        """
        This method udpates the weights and bias of one randomly selected neuron in a randomly selected
        hidden layer.

        NOTE: There are a couple of lines that are commented out. These lines were created for visualization purposes
              so that I could see if the neuron parameters were really updating. If you want to see those results,
              simply uncomment those lines and uncomment the necessary lines in the cell below this one. 
        """
        # Don't do anything if the network didn't initialize properly
        if self.template_network is None:
            print("Template network not initialized")
            return
        
        # Find indicies for all linear layers
        layer_indices = [i for i, layer in enumerate(self.template_network) if isinstance(layer, nn.Linear)]

        # Select random layer
        selected_index = random.choice(layer_indices) 
        linear = self.template_network[selected_index]

        # Select a random neuron and generate random weights/bias
        new_weights = torch.rand(linear.in_features)
        new_bias = torch.rand(1)
        neuron_index = random.choice(range(linear.out_features))

        #print("Newly generated weights and biases")
        #print(new_weights)
        #print(new_bias)
        
        layers = list(self.template_network) # Convert network to list

        #old_weights = layers[selected_index].weight[neuron_index].clone() 
        #old_bias = layers[selected_index].bias[neuron_index].clone() 

        # Update neuron weights and bias
        with torch.no_grad():
            layers[selected_index].weight[neuron_index] = new_weights
            layers[selected_index].bias[neuron_index] = new_bias

        self.template_network = nn.Sequential(*layers) # Updated network

        print(f"Updated neuron {neuron_index} in layer {selected_index}")

        #return selected_index, neuron_index, old_weights, old_bias 



In [ ]:
config = configparser.ConfigParser()
config.read('config.ini')
hyperparameters = getHyperParameters(config)

test_eons = EONS(hyperparameters)
test_eons.make_template_network(50, 10)
print(test_eons.template_network, end="\n\n")
test_eons.add_node()
print(test_eons.template_network, end="\n\n")
test_eons.remove_node()
print(test_eons.template_network, end="\n\n")
test_eons.update_node_param()

# UNCOMMENT THE LINES BELOW IF YOU WANT TO PRINT OUT RESULTS FROM UPDATE_NODE_PARAM() AND SEE UPDATES
# layer_index, neuron_index, old_weights, old_bias = test_eons.update_node_param() 
# print("\nWeights and Biases before the update")
# print(old_weights)
# print(old_bias)

# print("\nWeights and Biases after the update")
# layers = list(test_eons.template_network)
# print(layers[layer_index].weight[neuron_index])
# print(layers[layer_index].bias[neuron_index])

Updated neuron 1 in layer 2
